In [1]:
## 라이브러리 추가하기
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

## 트레이닝 필요한 파라메터를 설정하기
lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 네트워크 구축하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x

## 네트워크를 저장하거나 불러오는 함수 작성하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

## MNIST 데이터 불러오기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

## 네트워크 설정 및 필요한 손실함수 구현하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

## 트레이닝 시작하기
for epoch in range(1, num_epoch + 1):
    net.train()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader, 1):
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)


        optim.zero_grad()

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        loss.backward()

        optim.step()

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    writer.add_scalar('loss', np.mean(loss_arr), epoch)
    writer.add_scalar('acc', np.mean(acc_arr), epoch)

    save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)

writer.close()

TRAIN: EPOCH 0001/0010 | BATCH 0001/0938 | LOSS: 2.2945 | ACC 0.1094
TRAIN: EPOCH 0001/0010 | BATCH 0002/0938 | LOSS: 2.3011 | ACC 0.1250
TRAIN: EPOCH 0001/0010 | BATCH 0003/0938 | LOSS: 2.2943 | ACC 0.1094
TRAIN: EPOCH 0001/0010 | BATCH 0004/0938 | LOSS: 2.2997 | ACC 0.1133
TRAIN: EPOCH 0001/0010 | BATCH 0005/0938 | LOSS: 2.2941 | ACC 0.1219
TRAIN: EPOCH 0001/0010 | BATCH 0006/0938 | LOSS: 2.2936 | ACC 0.1224
TRAIN: EPOCH 0001/0010 | BATCH 0007/0938 | LOSS: 2.2925 | ACC 0.1272
TRAIN: EPOCH 0001/0010 | BATCH 0008/0938 | LOSS: 2.2925 | ACC 0.1270
TRAIN: EPOCH 0001/0010 | BATCH 0009/0938 | LOSS: 2.2939 | ACC 0.1285
TRAIN: EPOCH 0001/0010 | BATCH 0010/0938 | LOSS: 2.2891 | ACC 0.1328
TRAIN: EPOCH 0001/0010 | BATCH 0011/0938 | LOSS: 2.2857 | ACC 0.1435
TRAIN: EPOCH 0001/0010 | BATCH 0012/0938 | LOSS: 2.2817 | ACC 0.1432
TRAIN: EPOCH 0001/0010 | BATCH 0013/0938 | LOSS: 2.2793 | ACC 0.1454
TRAIN: EPOCH 0001/0010 | BATCH 0014/0938 | LOSS: 2.2752 | ACC 0.1529
TRAIN: EPOCH 0001/0010 | BATCH 001

TRAIN: EPOCH 0001/0010 | BATCH 0128/0938 | LOSS: 1.4562 | ACC 0.5089
TRAIN: EPOCH 0001/0010 | BATCH 0129/0938 | LOSS: 1.4515 | ACC 0.5109
TRAIN: EPOCH 0001/0010 | BATCH 0130/0938 | LOSS: 1.4451 | ACC 0.5132
TRAIN: EPOCH 0001/0010 | BATCH 0131/0938 | LOSS: 1.4397 | ACC 0.5153
TRAIN: EPOCH 0001/0010 | BATCH 0132/0938 | LOSS: 1.4346 | ACC 0.5168
TRAIN: EPOCH 0001/0010 | BATCH 0133/0938 | LOSS: 1.4286 | ACC 0.5186
TRAIN: EPOCH 0001/0010 | BATCH 0134/0938 | LOSS: 1.4250 | ACC 0.5199
TRAIN: EPOCH 0001/0010 | BATCH 0135/0938 | LOSS: 1.4213 | ACC 0.5215
TRAIN: EPOCH 0001/0010 | BATCH 0136/0938 | LOSS: 1.4153 | ACC 0.5240
TRAIN: EPOCH 0001/0010 | BATCH 0137/0938 | LOSS: 1.4097 | ACC 0.5260
TRAIN: EPOCH 0001/0010 | BATCH 0138/0938 | LOSS: 1.4062 | ACC 0.5273
TRAIN: EPOCH 0001/0010 | BATCH 0139/0938 | LOSS: 1.4007 | ACC 0.5293
TRAIN: EPOCH 0001/0010 | BATCH 0140/0938 | LOSS: 1.3954 | ACC 0.5309
TRAIN: EPOCH 0001/0010 | BATCH 0141/0938 | LOSS: 1.3908 | ACC 0.5327
TRAIN: EPOCH 0001/0010 | BATCH 014

TRAIN: EPOCH 0001/0010 | BATCH 0255/0938 | LOSS: 1.0511 | ACC 0.6518
TRAIN: EPOCH 0001/0010 | BATCH 0256/0938 | LOSS: 1.0499 | ACC 0.6522
TRAIN: EPOCH 0001/0010 | BATCH 0257/0938 | LOSS: 1.0484 | ACC 0.6528
TRAIN: EPOCH 0001/0010 | BATCH 0258/0938 | LOSS: 1.0460 | ACC 0.6537
TRAIN: EPOCH 0001/0010 | BATCH 0259/0938 | LOSS: 1.0442 | ACC 0.6544
TRAIN: EPOCH 0001/0010 | BATCH 0260/0938 | LOSS: 1.0426 | ACC 0.6550
TRAIN: EPOCH 0001/0010 | BATCH 0261/0938 | LOSS: 1.0409 | ACC 0.6558
TRAIN: EPOCH 0001/0010 | BATCH 0262/0938 | LOSS: 1.0388 | ACC 0.6564
TRAIN: EPOCH 0001/0010 | BATCH 0263/0938 | LOSS: 1.0368 | ACC 0.6571
TRAIN: EPOCH 0001/0010 | BATCH 0264/0938 | LOSS: 1.0344 | ACC 0.6580
TRAIN: EPOCH 0001/0010 | BATCH 0265/0938 | LOSS: 1.0325 | ACC 0.6586
TRAIN: EPOCH 0001/0010 | BATCH 0266/0938 | LOSS: 1.0303 | ACC 0.6595
TRAIN: EPOCH 0001/0010 | BATCH 0267/0938 | LOSS: 1.0287 | ACC 0.6601
TRAIN: EPOCH 0001/0010 | BATCH 0268/0938 | LOSS: 1.0261 | ACC 0.6610
TRAIN: EPOCH 0001/0010 | BATCH 026

TRAIN: EPOCH 0001/0010 | BATCH 0382/0938 | LOSS: 0.8526 | ACC 0.7218
TRAIN: EPOCH 0001/0010 | BATCH 0383/0938 | LOSS: 0.8516 | ACC 0.7221
TRAIN: EPOCH 0001/0010 | BATCH 0384/0938 | LOSS: 0.8502 | ACC 0.7226
TRAIN: EPOCH 0001/0010 | BATCH 0385/0938 | LOSS: 0.8487 | ACC 0.7230
TRAIN: EPOCH 0001/0010 | BATCH 0386/0938 | LOSS: 0.8478 | ACC 0.7234
TRAIN: EPOCH 0001/0010 | BATCH 0387/0938 | LOSS: 0.8463 | ACC 0.7240
TRAIN: EPOCH 0001/0010 | BATCH 0388/0938 | LOSS: 0.8453 | ACC 0.7242
TRAIN: EPOCH 0001/0010 | BATCH 0389/0938 | LOSS: 0.8441 | ACC 0.7247
TRAIN: EPOCH 0001/0010 | BATCH 0390/0938 | LOSS: 0.8432 | ACC 0.7249
TRAIN: EPOCH 0001/0010 | BATCH 0391/0938 | LOSS: 0.8421 | ACC 0.7252
TRAIN: EPOCH 0001/0010 | BATCH 0392/0938 | LOSS: 0.8414 | ACC 0.7256
TRAIN: EPOCH 0001/0010 | BATCH 0393/0938 | LOSS: 0.8406 | ACC 0.7259
TRAIN: EPOCH 0001/0010 | BATCH 0394/0938 | LOSS: 0.8394 | ACC 0.7264
TRAIN: EPOCH 0001/0010 | BATCH 0395/0938 | LOSS: 0.8384 | ACC 0.7267
TRAIN: EPOCH 0001/0010 | BATCH 039

TRAIN: EPOCH 0001/0010 | BATCH 0510/0938 | LOSS: 0.7441 | ACC 0.7598
TRAIN: EPOCH 0001/0010 | BATCH 0511/0938 | LOSS: 0.7435 | ACC 0.7600
TRAIN: EPOCH 0001/0010 | BATCH 0512/0938 | LOSS: 0.7431 | ACC 0.7602
TRAIN: EPOCH 0001/0010 | BATCH 0513/0938 | LOSS: 0.7425 | ACC 0.7604
TRAIN: EPOCH 0001/0010 | BATCH 0514/0938 | LOSS: 0.7419 | ACC 0.7606
TRAIN: EPOCH 0001/0010 | BATCH 0515/0938 | LOSS: 0.7410 | ACC 0.7609
TRAIN: EPOCH 0001/0010 | BATCH 0516/0938 | LOSS: 0.7408 | ACC 0.7610
TRAIN: EPOCH 0001/0010 | BATCH 0517/0938 | LOSS: 0.7398 | ACC 0.7614
TRAIN: EPOCH 0001/0010 | BATCH 0518/0938 | LOSS: 0.7389 | ACC 0.7616
TRAIN: EPOCH 0001/0010 | BATCH 0519/0938 | LOSS: 0.7381 | ACC 0.7619
TRAIN: EPOCH 0001/0010 | BATCH 0520/0938 | LOSS: 0.7379 | ACC 0.7621
TRAIN: EPOCH 0001/0010 | BATCH 0521/0938 | LOSS: 0.7372 | ACC 0.7624
TRAIN: EPOCH 0001/0010 | BATCH 0522/0938 | LOSS: 0.7365 | ACC 0.7627
TRAIN: EPOCH 0001/0010 | BATCH 0523/0938 | LOSS: 0.7359 | ACC 0.7630
TRAIN: EPOCH 0001/0010 | BATCH 052

...

TRAIN: EPOCH 0010/0010 | BATCH 0384/0938 | LOSS: 0.1479 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0385/0938 | LOSS: 0.1479 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0386/0938 | LOSS: 0.1479 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0387/0938 | LOSS: 0.1479 | ACC 0.9545
TRAIN: EPOCH 0010/0010 | BATCH 0388/0938 | LOSS: 0.1480 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0389/0938 | LOSS: 0.1480 | ACC 0.9545
TRAIN: EPOCH 0010/0010 | BATCH 0390/0938 | LOSS: 0.1482 | ACC 0.9543
TRAIN: EPOCH 0010/0010 | BATCH 0391/0938 | LOSS: 0.1480 | ACC 0.9544
TRAIN: EPOCH 0010/0010 | BATCH 0392/0938 | LOSS: 0.1482 | ACC 0.9544
TRAIN: EPOCH 0010/0010 | BATCH 0393/0938 | LOSS: 0.1484 | ACC 0.9542
TRAIN: EPOCH 0010/0010 | BATCH 0394/0938 | LOSS: 0.1485 | ACC 0.9542
TRAIN: EPOCH 0010/0010 | BATCH 0395/0938 | LOSS: 0.1486 | ACC 0.9542
TRAIN: EPOCH 0010/0010 | BATCH 0396/0938 | LOSS: 0.1484 | ACC 0.9542
TRAIN: EPOCH 0010/0010 | BATCH 0397/0938 | LOSS: 0.1483 | ACC 0.9543
TRAIN: EPOCH 0010/0010 | BATCH 039

TRAIN: EPOCH 0010/0010 | BATCH 0512/0938 | LOSS: 0.1485 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0513/0938 | LOSS: 0.1484 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0514/0938 | LOSS: 0.1483 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0515/0938 | LOSS: 0.1485 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0516/0938 | LOSS: 0.1486 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0517/0938 | LOSS: 0.1486 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0518/0938 | LOSS: 0.1487 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0519/0938 | LOSS: 0.1486 | ACC 0.9546
TRAIN: EPOCH 0010/0010 | BATCH 0520/0938 | LOSS: 0.1485 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0521/0938 | LOSS: 0.1486 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0522/0938 | LOSS: 0.1483 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0523/0938 | LOSS: 0.1484 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0524/0938 | LOSS: 0.1484 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 0525/0938 | LOSS: 0.1482 | ACC 0.9547
TRAIN: EPOCH 0010/0010 | BATCH 052

TRAIN: EPOCH 0010/0010 | BATCH 0639/0938 | LOSS: 0.1489 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0640/0938 | LOSS: 0.1489 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0641/0938 | LOSS: 0.1488 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0642/0938 | LOSS: 0.1488 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0643/0938 | LOSS: 0.1488 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0644/0938 | LOSS: 0.1490 | ACC 0.9551
TRAIN: EPOCH 0010/0010 | BATCH 0645/0938 | LOSS: 0.1493 | ACC 0.9551
TRAIN: EPOCH 0010/0010 | BATCH 0646/0938 | LOSS: 0.1491 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0647/0938 | LOSS: 0.1492 | ACC 0.9551
TRAIN: EPOCH 0010/0010 | BATCH 0648/0938 | LOSS: 0.1492 | ACC 0.9552
TRAIN: EPOCH 0010/0010 | BATCH 0649/0938 | LOSS: 0.1493 | ACC 0.9551
TRAIN: EPOCH 0010/0010 | BATCH 0650/0938 | LOSS: 0.1492 | ACC 0.9551
TRAIN: EPOCH 0010/0010 | BATCH 0651/0938 | LOSS: 0.1493 | ACC 0.9550
TRAIN: EPOCH 0010/0010 | BATCH 0652/0938 | LOSS: 0.1493 | ACC 0.9550
TRAIN: EPOCH 0010/0010 | BATCH 065

TRAIN: EPOCH 0010/0010 | BATCH 0767/0938 | LOSS: 0.1499 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0768/0938 | LOSS: 0.1499 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0769/0938 | LOSS: 0.1499 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0770/0938 | LOSS: 0.1499 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0771/0938 | LOSS: 0.1499 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0772/0938 | LOSS: 0.1498 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0773/0938 | LOSS: 0.1502 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0774/0938 | LOSS: 0.1503 | ACC 0.9548
TRAIN: EPOCH 0010/0010 | BATCH 0775/0938 | LOSS: 0.1502 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0776/0938 | LOSS: 0.1500 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0777/0938 | LOSS: 0.1501 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0778/0938 | LOSS: 0.1501 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 0779/0938 | LOSS: 0.1501 | ACC 0.9550
TRAIN: EPOCH 0010/0010 | BATCH 0780/0938 | LOSS: 0.1501 | ACC 0.9549
TRAIN: EPOCH 0010/0010 | BATCH 078

TRAIN: EPOCH 0010/0010 | BATCH 0896/0938 | LOSS: 0.1493 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0897/0938 | LOSS: 0.1493 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0898/0938 | LOSS: 0.1493 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0899/0938 | LOSS: 0.1495 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0900/0938 | LOSS: 0.1494 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0901/0938 | LOSS: 0.1494 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0902/0938 | LOSS: 0.1497 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0903/0938 | LOSS: 0.1497 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0904/0938 | LOSS: 0.1496 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0905/0938 | LOSS: 0.1497 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0906/0938 | LOSS: 0.1496 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0907/0938 | LOSS: 0.1496 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0908/0938 | LOSS: 0.1496 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 0909/0938 | LOSS: 0.1497 | ACC 0.9554
TRAIN: EPOCH 0010/0010 | BATCH 091

In [2]:
## 라이브러리 추가하기
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

## 트레이닝 필요한 파라메터를 설정하기
lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## 네트워크 구축하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x


## 네트워크를 저장하거나 불러오는 함수 작성하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))


def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim


## MNIST 데이터 불러오기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=False, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

## 네트워크 설정 및 필요한 손실함수 구현하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

net, optim = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

## 트레이닝 시작하기
with torch.no_grad():
    # net.train()
    net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader, 1):
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TEST: BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))


TEST: BATCH 0001/0157 | LOSS: 0.0037 | ACC 1.0000
TEST: BATCH 0002/0157 | LOSS: 0.0056 | ACC 1.0000
TEST: BATCH 0003/0157 | LOSS: 0.0054 | ACC 1.0000
TEST: BATCH 0004/0157 | LOSS: 0.0098 | ACC 0.9961
TEST: BATCH 0005/0157 | LOSS: 0.0201 | ACC 0.9938
TEST: BATCH 0006/0157 | LOSS: 0.0269 | ACC 0.9896
TEST: BATCH 0007/0157 | LOSS: 0.0287 | ACC 0.9888
TEST: BATCH 0008/0157 | LOSS: 0.0388 | ACC 0.9863
TEST: BATCH 0009/0157 | LOSS: 0.0359 | ACC 0.9878
TEST: BATCH 0010/0157 | LOSS: 0.0366 | ACC 0.9875
TEST: BATCH 0011/0157 | LOSS: 0.0355 | ACC 0.9886
TEST: BATCH 0012/0157 | LOSS: 0.0355 | ACC 0.9896
TEST: BATCH 0013/0157 | LOSS: 0.0333 | ACC 0.9904
TEST: BATCH 0014/0157 | LOSS: 0.0321 | ACC 0.9911
TEST: BATCH 0015/0157 | LOSS: 0.0384 | ACC 0.9896
TEST: BATCH 0016/0157 | LOSS: 0.0427 | ACC 0.9883
TEST: BATCH 0017/0157 | LOSS: 0.0443 | ACC 0.9871
TEST: BATCH 0018/0157 | LOSS: 0.0452 | ACC 0.9861
TEST: BATCH 0019/0157 | LOSS: 0.0453 | ACC 0.9860
TEST: BATCH 0020/0157 | LOSS: 0.0486 | ACC 0.9844


In [3]:
## 라이브러리 추가하기
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt

from torchvision import transforms, datasets

## 트레이닝 파라메터 설정하기
lr = 1e-3
batch_size = 4
num_epoch = 100

data_dir = './datasets'
ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 네트워크 구축하기
class  UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True):
            layers = []
            layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                                 kernel_size=kernel_size, stride=stride,
                                 bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_channels)]
            layers += [nn.ReLU()]

            cbr = nn.Sequential(*layers)

            return cbr

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=1, out_channels=64)
        self.enc1_2 = CBR2d(in_channels=64, out_channels=64)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=64, out_channels=128)
        self.enc2_2 = CBR2d(in_channels=128, out_channels=128)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=128, out_channels=256)
        self.enc3_2 = CBR2d(in_channels=256, out_channels=256)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=256, out_channels=512)
        self.enc4_2 = CBR2d(in_channels=512, out_channels=512)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=512, out_channels=1024)

        # Expansive path
        self.dec5_1 = CBR2d(in_channels=1024, out_channels=512)

        self.unpool4 = nn.ConvTranspose2d(in_channels=512, out_channels=512,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 512, out_channels=512)
        self.dec4_1 = CBR2d(in_channels=512, out_channels=256)

        self.unpool3 = nn.ConvTranspose2d(in_channels=256, out_channels=256,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2 * 256, out_channels=256)
        self.dec3_1 = CBR2d(in_channels=256, out_channels=128)

        self.unpool2 = nn.ConvTranspose2d(in_channels=128, out_channels=128,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 128, out_channels=128)
        self.dec2_1 = CBR2d(in_channels=128, out_channels=64)

        self.unpool1 = nn.ConvTranspose2d(in_channels=64, out_channels=64,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 64, out_channels=64)
        self.dec1_1 = CBR2d(in_channels=64, out_channels=64)

        self.fc = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)


        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_1), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        x = self.fc(dec1_1)

        return x

## 데이터 로더를 구현하기
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform =transform

        lst_data = os.listdir(self.data_dir)

        lst_label = [f for f in lst_data if f.startswith('label')]
        lst_input = [f for f in lst_data if f.startswith('input')]

        lst_label.sort()
        lst_input.sort()

        self.lst_label = lst_label
        self.lst_input = lst_input

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        label = label/255.0
        input = input/255.0

        if label.ndim == 2:
            label = label[:, :, np.newaxis]
        if label.ndim == 2:
            input = input[:, :, np.newaxis]

        data = {'input': input, 'label': label}

        if self.transform:
            data = self.transform(data)

        return data


# ##
# dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'))
#
# ##
# data = dataset_train.__getitem__(0)
#
# input = data['input']
# label = data['label']
#
# ##
# plt.subplot(121)
# plt.imshow(input.squeeze())
#
# plt.subplot(122)
# plt.imshow(label.squeeze())
#
# plt.show()

## 트랜스폼 구현하기

class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = label.transpose((2, 0, 1)).astype(np.float32)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data
class Nomalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        label, input = data['label'], data['input']

        input = (input - self.mean) / self.std

        data = {'label': label, 'input': input}

        return data

class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            label = np.fliplr(label)
            input = np.fliplr(input)

        if np.random.rand() > 0.5:
            label = np.flipud(label)
            input = np.flipud(input)

        data = {'label': label, 'input': input}

        return data

## 네트워크 학습하기
transform = transforms.Compose([Nomalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])

dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=transform)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=8)

dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=transform)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=8)

## 네트워크 생성하기
net = UNet().to(device)

## 손실함수 정의하기
fn_loss = nn.BCEWithLogitsLoss().to(device)

## Optimizer 설정하기
optim = torch.optim.Adam(net.parameters(), lr=lr)

## 그밖에 부수적인 variables 설정하기
num_data_train = len(dataset_train)
num_data_val = len(dataset_val)

num_batch_train = np.ceil(num_data_train / batch_size)
num_batch_val = np.ceil(num_data_val / batch_size)

## 그밖에 부수적인 functions 설정하기
fn_tonumpy = lambda x: x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std: (x * std) + mean
fn_class = lambda x: 1.0 * (x > 0.5)

## Tensorboard 를 사용하기 위한 SummaryWriter 설정
writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

## 네트워크 저장하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               "./%s/model_epoch%d.pth" % (ckpt_dir, epoch))

## 네트워크 불러오기
def load(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir):
        epoch = 0
        return net, optim, epoch

    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('pth')[0])

    return net, optim, epoch

## 네트워크 학습시키기
st_epoch = 0
net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

for epoch in range(st_epoch + 1, num_epoch + 1):
    net.train()
    loss_arr = []

    for batch, data in enumerate(loader_train, 1):
        label = data['label'].to(device)
        input = data['input'].to(device)

        output = net(input)

        # backward pass
        optim.zero_grad()

        loss = fn_loss(output, label)
        loss.backward()

        optim.step()

        # 손실함수 계산
        loss_arr += [loss.item()]

        print("TRAIN: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
              (epoch, num_epoch, batch, num_batch_train, np.mean(loss_arr)))

        # Tensorboard 저장하기
        label = fn_tonumpy(label)
        input = fn_tonumpy(fn_denorm(input, mean=0.5, std=0.5))
        output = fn_tonumpy(fn_class(output))

        writer_train.add_image('label', label, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', input, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_batch_train * (epoch - 1) + batch, dataformats='NHWC')

    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)


    with torch.no_grad():
        net.eval()
        loss_arr = []

        for batch, data in enumerate(loader_val, 1):
            # forward pass
            label = data['label'].to(device)
            input = data['input'].to(device)

            output = net(input)

            #손실함수 계산하기
            loss = fn_loss(output, label)

            loss_arr += [loss.item()]

            print("VALID: EPOCH %04d / %04d | BATCH %04d / %04d | LOSS %.4f" %
                  (epoch, num_epoch, batch, num_batch_val, np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            input = fn_tonumpy(fn_denorm(input))
            output = fn_tonumpy(fn_class(output))

            writer_val.add_image('label', label, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', input, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_batch_val * (epoch - 1) + batch, dataformats='NHWC')

    writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

    if epoch % 50 == 0:
        save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch)

writer_train.close()
writer_val.close()

RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "enc1_1.0.weight", "enc1_1.0.bias", "enc1_1.1.weight", "enc1_1.1.bias", "enc1_1.1.running_mean", "enc1_1.1.running_var", "enc1_2.0.weight", "enc1_2.0.bias", "enc1_2.1.weight", "enc1_2.1.bias", "enc1_2.1.running_mean", "enc1_2.1.running_var", "enc2_1.0.weight", "enc2_1.0.bias", "enc2_1.1.weight", "enc2_1.1.bias", "enc2_1.1.running_mean", "enc2_1.1.running_var", "enc2_2.0.weight", "enc2_2.0.bias", "enc2_2.1.weight", "enc2_2.1.bias", "enc2_2.1.running_mean", "enc2_2.1.running_var", "enc3_1.0.weight", "enc3_1.0.bias", "enc3_1.1.weight", "enc3_1.1.bias", "enc3_1.1.running_mean", "enc3_1.1.running_var", "enc3_2.0.weight", "enc3_2.0.bias", "enc3_2.1.weight", "enc3_2.1.bias", "enc3_2.1.running_mean", "enc3_2.1.running_var", "enc4_1.0.weight", "enc4_1.0.bias", "enc4_1.1.weight", "enc4_1.1.bias", "enc4_1.1.running_mean", "enc4_1.1.running_var", "enc4_2.0.weight", "enc4_2.0.bias", "enc4_2.1.weight", "enc4_2.1.bi